In [ ]:
%pylab inline
import matplotlib
import matplotlib.pyplot as plt
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from functions import plot,calc
from netCDF4 import Dataset
from wrf import getvar, vinterp, smooth2d
import numpy as np
from matplotlib.cm import get_cmap
from matplotlib.ticker import MultipleLocator
import matplotlib.transforms as mtransforms
import warnings
warnings.filterwarnings("ignore")
import matplotlib.transforms as mtransforms


ti = 0
rol = -1000
dx = 4
dz = 0.2
nc_ctl_48h = Dataset('/p/work1/lloveras/adj_4km/long_runs/ctl_48h/wrfout_d01_2021-01-03_00_00_00')
nc_adj_48h = Dataset('/p/work1/lloveras/adj_4km/long_runs/adj_48h/wrfout_d01_2021-01-03_00_00_00')
nc_wave_48h = Dataset('/p/work1/lloveras/adj_4km/long_runs/wave_48h/wrfout_d01_2021-01-03_00_00_00')


In [ ]:
slp_ctl_48h = np.asarray(getvar(nc_ctl_48h,'slp',timeidx=ti))
dbz_ctl_48h = np.asarray(getvar(nc_ctl_48h,'mdbz',timeidx=ti))
t_ctl_48h = np.asarray(getvar(nc_ctl_48h,'th',timeidx=ti))
t_adj_48h = np.asarray(getvar(nc_adj_48h,'th',timeidx=ti))
t_wave_48h = np.asarray(getvar(nc_wave_48h,'th',timeidx=ti))


In [ ]:
x1_48 = 300
x2_48 = 801
y1_48 = 625
y2_48 = 1001
y_cross_adj48 = 830
y_cross_wave48 = 765

z1 = 0
z2 = 41
zlev_adj = 15
zlev_wave = 5

yax1_48 = np.arange(y1_48*dx,y2_48*dx,500).astype(int)
yax2_48 = (yax1_48 - y1_48*dx)/float(dx)
yax2_48 = yax2_48.astype(int)
yax1_48 = ["%0.0f" % number for number in yax1_48]
xax1_48 = np.arange(x1_48*dx,x2_48*dx,500).astype(int)
xax2_48 = (xax1_48 - x1_48*dx)/float(dx)
xax2_48 = xax2_48.astype(int)
xax1_48 = ["%0.0f" % number for number in xax1_48]
zax1 = np.arange(z1*dz,z2*dz,2).astype(int)
zax2 = (zax1 - z1*dz)/float(dz)
zax2 = zax2.astype(int)
xax1_cross_48 = np.arange(x1_48*dx,x2_48*dx,100).astype(int)
xax2_cross_48 = (xax1_cross_48 - x1_48*dx)/float(dx)
xax2_cross_48 = xax2_cross_48.astype(int)
xax1_cross_48 = ["%0.0f" % number for number in xax1_cross_48]

dt_adj_48h = t_adj_48h - t_ctl_48h
dt_wave_48h = t_wave_48h - t_ctl_48h

pplot_48h = np.roll(slp_ctl_48h,rol,axis=-1)[y1_48:y2_48,x1_48:x2_48]
dbzplot_48h = np.roll(dbz_ctl_48h,rol,axis=-1)[y1_48:y2_48,x1_48:x2_48]
tplot_adj_48h = np.roll(dt_adj_48h,rol,axis=-1)[zlev_adj,y1_48:y2_48,x1_48:x2_48]
dtcross_adj_48h = np.roll(dt_adj_48h,rol,axis=-1)[z1:z2,y_cross_adj48,x1_48:x2_48]
tplot_wave_48h = np.roll(dt_wave_48h,rol,axis=-1)[zlev_wave,y1_48:y2_48,x1_48:x2_48]
dtcross_wave_48h = np.roll(dt_wave_48h,rol,axis=-1)[z1:z2,y_cross_wave48,x1_48:x2_48]

In [ ]:
fig, axd = plt.subplot_mosaic([['top left','top right','cbar'],
                               ['bottom left','bottom right','cbar']],
                              constrained_layout=True, figsize=(15.5,9.5), 
                              gridspec_kw={'width_ratios':[1,1,0.075], 'height_ratios':[1,0.8]})

#lvls = np.asarray([-1,-0.8,-0.6,-0.4,-0.2,0.2,0.4,0.6,0.8,1.0])
lvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/10
cmap = plt.get_cmap('bwr')
cmap.set_under('navy')
cmap.set_over('brown')
trans = mtransforms.ScaledTranslation(1/10, -1/10, fig.dpi_scale_trans)

cs48 = axd['top left'].contour(pplot_48h,colors='k',levels=np.arange(952,1060,2),linewidths=2,alpha=0.9)
axd['top left'].clabel(cs48,fmt='%1.0f',inline=1,levels=np.arange(952,1060,4),fontsize=10,colors='k')
im1 = axd['top left'].contourf(tplot_adj_48h,levels=lvls,extend='both',cmap=get_cmap('bwr'),alpha=0.9)
axd['top left'].contourf(dbzplot_48h,levels=[0,0.01],colors='k',extend='max',alpha=0.25)
axd['top left'].axhline(y=y_cross_adj48 - y1_48,color='k',linestyle='dashed',alpha=0.6)
axd['top left'].set_xticks(xax2_48)
axd['top left'].set_xticklabels(['','','','',''])
axd['top left'].set_ylabel('South-North (km)',labelpad=6,fontsize=20)
axd['top left'].set_yticks(yax2_48)
axd['top left'].set_yticklabels(yax1_48)
axd['top left'].tick_params(axis='both', labelsize=20, which='major', pad=8,length=8)
axd['top left'].set_aspect('equal')
axd['top left'].grid(alpha=0.5)
axd['top left'].set_title('Adjoint',pad=12,fontsize=24)
axd['top left'].text(0.0, 1.0, '(a)',transform=axd['top left'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))

cs248 = axd['top right'].contour(pplot_48h,colors='k',levels=np.arange(952,1060,2),linewidths=2,alpha=0.9)
axd['top right'].clabel(cs248,fmt='%1.0f',inline=1,levels=np.arange(952,1060,4),fontsize=10,colors='k')
im2 = axd['top right'].contourf(tplot_wave_48h,levels=lvls,extend='both',cmap=get_cmap('bwr'),alpha=0.9)
axd['top right'].contourf(dbzplot_48h,levels=[0,0.01],colors='k',extend='max',alpha=0.25)
axd['top right'].axhline(y=y_cross_wave48 - y1_48,color='k',linestyle='dashed',alpha=0.6)
axd['top right'].set_xticks(xax2_48)
axd['top right'].set_xticklabels(['','','','',''])
axd['top right'].set_yticks(yax2_48)
axd['top right'].set_yticklabels(['','','',''])
axd['top right'].tick_params(axis='both', labelsize=20, which='major', pad=8,length=8)
axd['top right'].set_aspect('equal')
axd['top right'].grid(alpha=0.5)
axd['top right'].set_title('Wave',pad=12,fontsize=24)
axd['top right'].text(0.0, 1.0, '(b)',transform=axd['top right'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))

im3 = axd['bottom left'].contourf(dtcross_adj_48h,levels=lvls,extend='both',cmap=get_cmap('bwr'),alpha=0.9)
axd['bottom left'].set_xlabel('West-East (km)',fontsize=20,labelpad=6)
axd['bottom left'].set_ylabel('Height (km)',labelpad=10,fontsize=20)
axd['bottom left'].axhline(y=zlev_adj,color='k',linestyle='dashed',alpha=0.6)
axd['bottom left'].set_xticks(xax2_48)
axd['bottom left'].set_xticklabels(xax1_48)
axd['bottom left'].set_yticks(zax2)
axd['bottom left'].set_yticklabels(zax1)
axd['bottom left'].grid(alpha=0.5)
axd['bottom left'].tick_params(axis='both', which='major', labelsize=20, pad=8,length=8)
axd['bottom left'].text(0.0, 1.0, '(c)',transform=axd['bottom left'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))

im4 = axd['bottom right'].contourf(dtcross_wave_48h,levels=lvls,extend='both',cmap=get_cmap('bwr'),alpha=0.9)
axd['bottom right'].set_xlabel('West-East (km)',fontsize=20,labelpad=6)
axd['bottom right'].axhline(y=zlev_wave,color='k',linestyle='dashed',alpha=0.6)
axd['bottom right'].set_xticks(xax2_48)
axd['bottom right'].set_xticklabels(xax1_48)
axd['bottom right'].set_yticks(zax2)
axd['bottom right'].set_yticklabels(['','','','',''])
axd['bottom right'].grid(alpha=0.5)
axd['bottom right'].tick_params(axis='both', which='major', labelsize=20, pad=8,length=8)
axd['bottom right'].text(0.0, 1.0, '(d)',transform=axd['bottom right'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))

cbar1 = fig.colorbar(im1, orientation="vertical", cax=axd['cbar'])
cbar1.ax.tick_params(labelsize=20, pad=10)

plt.savefig('/p/work1/lloveras/adj_4km/figs/fig03_perts.pdf',bbox_inches='tight')